In [2]:
##do_runcode
##%overwritefile
##%file:src/do_c_runcode.py
##%noruncode
    def do_runcode(self,return_code,file_name,magics,code, silent, store_history=True,
                    user_expressions=None, allow_stdin=True):
        return_code=return_code
        file_name=file_name
        bcancel_exec=False
        retinfo=self.mymagics.get_retinfo()
        retstr=''
        ##代码运行前
        ################# repl mode run code files
        #FIXME:
        if len(self.mymagics.addkey2dict(magics,'execfile'))>0:
            file_name=magics['execfile']
        if magics['_st']['runmode']=='repl':
            self.mymagics._start_replprg(file_name,magics['_st']['args'],magics)

            return_code=self.replwrapper.child.status
            bcancel_exec,retstr=self.mymagics.raise_plugin(code,magics,return_code,file_name,3,2)
            return bcancel_exec,retinfo,magics, code,file_name,retstr
        ############################################

    ############################################
        p=None
        #################dynamically load and execute code
        #FIXME:
        
        if len(magics['dlrun'])>0:
            p = self.mymagics.create_jupyter_subprocess([self.mymagics.master_path, file_name] + magics['_st']['args'],env=self.mymagics.addkey2dict(magics,'env'),magics=magics)
        #################
        else:
            p = self.mymagics.create_jupyter_subprocess([file_name] + magics['_st']['args'],env=self.mymagics.get_magicsbykey(magics,'env'),magics=magics)
        self.mymagics.subprocess=p
        self.mymagics.g_rtsps[str(p.pid)]=p
        return_code=p.returncode
        ##代码启动后
        bcancel_exec,retstr=self.mymagics.raise_plugin(code,magics,return_code,file_name,3,2)
        # if bcancel_exec:return bcancel_exec,retinfo,magics, code,file_name,retstr
        
        if len(self.mymagics.addkey2dict(magics,'showpid'))>0:
            self.mymagics._logln("The process PID:"+str(p.pid))
        p.wait_end(magics)
        return_code=p.returncode
        ##代码运行结束
        # now remove the files we have just created
        # if(os.path.exists(source_file.name)):
            # os.remove(source_file.name)
        # if(os.path.exists(binary_filename)):
            # os.remove(binary_filename)

        # if p.returncode != 0:
            # self._write_to_stderr("[C kernel] Executable exited with code {}".format(p.returncode))
        return bcancel_exec,retinfo,magics, code,file_name,retstr

[MyPythonKernel183521] Info:file /mnt/h/Jupyter/Myjupyter-kernel/kernel/C/src/do_c_runcode.py created successfully


In [3]:
##do_compile_code
##%overwritefile
##%file:src/do_c_compilecode.py
##%noruncode
    def do_compile_code(self,return_code,file_name,magics,code, silent, store_history=True,
                    user_expressions=None, allow_stdin=True):
        return_code=0
        file_name=file_name
        sourcefilename=file_name
        bcancel_exec=False
        retinfo=self.mymagics.get_retinfo()
        retstr=''

        returncode,binary_filename=self._exec_gcc_(file_name,magics)
        file_name=binary_filename
        return_code=returncode
        
        if returncode!=0:return  True,retinfo, code,file_name,retstr

        return bcancel_exec,retinfo,magics, code,file_name,retstr

[MyPythonKernel183521] Info:file /mnt/h/Jupyter/Myjupyter-kernel/kernel/C/src/do_c_compilecode.py created successfully


In [4]:
##do_create_codefile
##%overwritefile
##%file:src/do_c_create_codefile.py
##%noruncode
    def do_create_codefile(self,magics,code, silent, store_history=True,
                    user_expressions=None, allow_stdin=True):
        return_code=0
        file_name=''
        sourcefilename=''
        bcancel_exec=False
        retinfo=self.mymagics.get_retinfo()
        retstr=''

        source_file=self.mymagics.create_codetemp_file(magics,code,suffix='.c')
        sourcefilename=source_file.name 
        newsrcfilename=source_file.name
        file_name=newsrcfilename
        return_code=True

        return  bcancel_exec,self.mymagics.get_retinfo(),magics, code,file_name,retstr

[MyPythonKernel183521] Info:file /mnt/h/Jupyter/Myjupyter-kernel/kernel/C/src/do_c_create_codefile.py created successfully


In [5]:
##do_preexecute
##%overwritefile
##%file:src/do_c_preexecute.py
##%noruncode
    def do_preexecute(self,code,magics, silent, store_history=True,
                user_expressions=None, allow_stdin=False):
        bcancel_exec=False
        retinfo=self.mymagics.get_retinfo()
        ############# run gdb and send command begin
        if len(self.mymagics.get_magicsbykey(magics,'rungdb'))>0:
            bcancel_exec=True
            retinfo= self.mymagics.replgdb_sendcmd(code,silent, store_history,
                user_expressions, allow_stdin)
            return bcancel_exec,retinfo,magics, code
        ############# run gdb and send command
        #############send replcmd's command
        if magics['_st']['runmode']=='repl':
            if hasattr(self, 'replcmdwrapper'):
                if self.replcmdwrapper :
                    bcancel_exec=True
                    retinfo= self.mymagics.repl_sendcmd(code, silent, store_history,
                        user_expressions, allow_stdin,magics)
                    return bcancel_exec,retinfo,magics, code
        if len(self.mymagics.addkey2dict(magics,'noruncode'))<1 :
            magics, code = self.mymagics._add_main(magics, code)
        return bcancel_exec,retinfo,magics, code

[MyPythonKernel183521] Info:file /mnt/h/Jupyter/Myjupyter-kernel/kernel/C/src/do_c_preexecute.py created successfully


In [6]:
## new kernel.py
##%overwritefile
##%file:../../../jupyter-MyC-kernel/jupyter_MyC_kernel/kernel.py
###%file:c_kernel.py
##%noruncode
#
#   MyC Jupyter Kernel 
#   generated by MyPython
#
##%include:../../src/head.py
from .MyKernel import MyKernel

class CKernel(MyKernel):
    implementation = 'jupyter-MyC-kernel'
    implementation_version = '1.0'
    language = 'C'
    language_version = ''
    language_info = {'name': 'text/x-csrc',
                     'mimetype': 'text/x-csrc',
                     'file_extension': '.c'}
    runfiletype='exe'
    banner = "C kernel.\n" \
             "Uses gcc, compiles in C11, and creates source code files and executables in temporary folder.\n"

    main_head = "#include <stdio.h>\n" \
            "#include <math.h>\n" \
            "int main(int argc, char* argv[], char** env){\n"

    main_foot = "\nreturn 0;\n}"

    def __init__(self, *args, **kwargs):
        super(CKernel, self).__init__(*args, **kwargs)
        self.runfiletype='script'
        self.kernelinfo="[MyCKernel{0}]".format(time.strftime("%H%M%S", time.localtime()))
        
        self.readOnlyFileSystem = False
        self.bufferedOutput = True
        self.linkMaths = True # always link math library
        self.wAll = True # show all warnings by default
        self.wError = False # but keep comipiling for warnings
#################
##%include:src/compile_with_gcc.py
##%include:src/c_exec_gcc_.py

##%include:src/do_c_runcode.py
##%include:src/do_c_compilecode.py
##%include:src/do_c_create_codefile.py
##%include:src/do_c_preexecute.py



[MyPythonKernel183521] Info:file /mnt/h/Jupyter/Myjupyter-kernel/kernel/C/../../../jupyter-MyC-kernel/jupyter_MyC_kernel/kernel.py created successfully
